
show timeline history?

ROUTE = /timeline

ROOT = true

TODO: swap out military connections for artists or celebrities

TODO: reflect expansions of empires or ideas with bubble

TODO: where vikings has explored through history

TODO: show aggregate counts of military personel



In [ ]:
const mustache = require('mustache')


const locations = [
  { name: "New York", lat: 40.7128, lng: -74.006 },
  { name: "London", lat: 51.5074, lng: -0.1278 },
  { name: "Paris", lat: 48.8566, lng: 2.3522 },
  { name: "Tokyo", lat: 35.6895, lng: 139.6917 }
];

const occupations = JSON.parse(importer.interpret('occupations.json').code)


function showTimeline() {
  return mustache.render(importer.interpret('timeline template').code, {
    LOCATIONS: JSON.stringify(locations),
    TIMESTAMP: Date.now(),
  })
}

module.exports = showTimeline



timeline template?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>World Doily</title>
  <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
  <link rel="stylesheet" href="/timeline-styles.css">
  <script>let locations = JSON.parse('{{{LOCATIONS}}}');</script>
</head>

<body>
  <div id="map"></div>

  <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
  <script src="/timeline-script.js?t={{{TIMESTAMP}}}"></script>
</body>

</html>


timeline-styles.css?

ROUTE = /timeline-styles.css


In [ ]:
body {
  font-family: sans-serif;
  margin: 0;
  padding: 0;
  text-align: center;
}

#map {
  height: 100%;
  width: 100%;
  position: fixed;
  
}



timeline-script.js?

ROUTE = /timeline-script.js


In [ ]:

const map = L.map('map', {
  maxBoundsViscosity: 1.0
}).setView([20, 0], 2);

const worldBounds = L.latLngBounds(L.latLng(-180, -Infinity),
  L.latLng(180, Infinity));

const minZoom = map.getBoundsZoom(worldBounds, true); // true = inside
map.setMinZoom(minZoom);
map.setMaxBounds(worldBounds);
map.options.maxBoundsViscosity = 1.0;

//L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
//}).addTo(map);

const mainLayer = L.tileLayer('https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png', {
  minZoom: 3,
  maxZoom: 17,
  attribution: '&copy; <a href="https://carto.com/">carto.com</a> contributors'
});
mainLayer.addTo(map);

let markerGroup = L.layerGroup().addTo(map);
let lineGroup = L.layerGroup().addTo(map);

function redrawMarkersAndLines() {
  markerGroup.clearLayers();
  lineGroup.clearLayers();

  const center = map.getCenter();
  const centerWrap = Math.round(center.lng / 360);
  const repeats = [centerWrap - 1,
    centerWrap,
  centerWrap + 1];

  repeats.forEach(r => {
    const latlngs = [];

    locations.forEach(loc => {
      const lngWrapped = loc.lng + (r * 360);
      const marker = L.marker([loc.lat, lngWrapped]);
      if (r === centerWrap) marker.bindPopup(loc.name); // Only bind popup on center
      marker.addTo(markerGroup);
      latlngs.push([loc.lat, lngWrapped]);
    });

    if (latlngs.length > 1) {
      L.polyline(latlngs, {
        color: 'blue'
      }).addTo(lineGroup);
    }
  })

}


function updateBoundsRequest() {
  const bounds = map.getBounds();
  const sw = bounds.getSouthWest(); // southwest corner
  const ne = bounds.getNorthEast(); // northeast corner

  fetch('/countries.json?t=' + Date.now() + '&bounds[0]=' + sw.lng + '&bounds[1]=' + sw.lat + '&bounds[2]=' + ne.lng + '&bounds[3]=' + ne.lat)
    .then(r => r.json()).then(results => {
      locations = results
      redrawMarkersAndLines()
    })
  redrawMarkersAndLines
}

updateBoundsRequest()


// Redraw when map finishes moving
map.on('moveend', () => {
  updateBoundsRequest();
  redrawMarkersAndLines();
})
// Initial draw
redrawMarkersAndLines();



occupations.json?


In [ ]:
[
  {
    "armyName": "British Army",
    "year": 1801,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Egypt"
  },
  {
    "armyName": "Napoleonic French Army",
    "year": 1807,
    "originalCountry": "France",
    "occupationCountry": "Spain"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1812,
    "originalCountry": "Russia",
    "occupationCountry": "Poland"
  },
  {
    "armyName": "British Army",
    "year": 1815,
    "originalCountry": "United Kingdom",
    "occupationCountry": "South Africa"
  },
  {
    "armyName": "Ottoman Army",
    "year": 1821,
    "originalCountry": "Ottoman Empire",
    "occupationCountry": "Greece"
  },
  {
    "armyName": "French Army",
    "year": 1830,
    "originalCountry": "France",
    "occupationCountry": "Algeria"
  },
  {
    "armyName": "British East India Company",
    "year": 1843,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Sindh (Pakistan)"
  },
  {
    "armyName": "United States Army",
    "year": 1846,
    "originalCountry": "United States",
    "occupationCountry": "California (Mexico)"
  },
  {
    "armyName": "Austrian Imperial Army",
    "year": 1849,
    "originalCountry": "Austria",
    "occupationCountry": "Hungary"
  },
  {
    "armyName": "French Army",
    "year": 1853,
    "originalCountry": "France",
    "occupationCountry": "Crimea"
  },
  {
    "armyName": "British Army",
    "year": 1858,
    "originalCountry": "United Kingdom",
    "occupationCountry": "India"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1864,
    "originalCountry": "Russia",
    "occupationCountry": "Caucasus (Chechnya, Dagestan)"
  },
  {
    "armyName": "United States Army",
    "year": 1865,
    "originalCountry": "United States",
    "occupationCountry": "Southern U.S. (Reconstruction)"
  },
  {
    "armyName": "French Army",
    "year": 1867,
    "originalCountry": "France",
    "occupationCountry": "Mexico"
  },
  {
    "armyName": "Prussian Army",
    "year": 1871,
    "originalCountry": "Germany",
    "occupationCountry": "Alsace-Lorraine (France)"
  },
  {
    "armyName": "British Army",
    "year": 1878,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Cyprus"
  },
  {
    "armyName": "Russian Imperial Army",
    "year": 1878,
    "originalCountry": "Russia",
    "occupationCountry": "Bulgaria"
  },
  {
    "armyName": "German Army",
    "year": 1884,
    "originalCountry": "Germany",
    "occupationCountry": "Namibia (South West Africa)"
  },
  {
    "armyName": "Belgian Army",
    "year": 1885,
    "originalCountry": "Belgium",
    "occupationCountry": "Congo Free State"
  },
  {
    "armyName": "Italian Army",
    "year": 1890,
    "originalCountry": "Italy",
    "occupationCountry": "Eritrea"
  },
  {
    "armyName": "British Army",
    "year": 1898,
    "originalCountry": "United Kingdom",
    "occupationCountry": "Sudan"
  }
]



lat/lon provider?

ROUTE = /countries.json


In [ ]:
const fs = require("fs");
const path = require('path')
const Papa = require("papaparse");

const TIMELINES = path.join(path.dirname(__dirname), 'Resources', 'Projects', 'timelines');
const CACHE = {}
const LOCATIONS = {
  'countries.csv': ["abbrv", "lat", "lng", "country"],
  'worldcities.csv': true,
  'places.csv': true,
}

// Load once at startup
function geolocationProvider(places, bounds) {
  //if (!places) {
  //  throw new Error("Query param 'places' is required")
  //}

  places = (places || '').split(",").map(c => c.trim().toLowerCase());
  isPlacesEmtpy = !places || places.length == 0 || (places.length == 1 && places[0].length == 0)
  const year = new Date().getFullYear();
  let isTooBig = true
  let isStillTooBig = true
  if(bounds) {
    bounds = normalizeBoundsToVisibleWorld(bounds)
    if((bounds[2] - bounds[0]) / 360 < Math.abs(0.05)) {
      isTooBig = false
      console.log('Not too big.')
    }
    if((bounds[2] - bounds[0]) / 360 < Math.abs(0.02)) {
      isStillTooBig = false
      console.log('Not still too big.')
    }
    console.log((bounds[2] - bounds[0]) / 360, bounds)
  }
  //console.log(bounds)

  let allPlaces = []
  let timelines = Object.keys(LOCATIONS)
  for (let i = 0; i < timelines.length; i++) {

    const results = loadCSV(path.join(TIMELINES, timelines[i]))
      .filter(p => isPlacesEmtpy || places.includes(p.country.toLowerCase()))
      .filter(p => !bounds || placeWithinBounds(p, bounds))
      .map(p => ({
        name: p.title || p.city_ascii || p.country,
        lat: p.representative_latitude || p.lat,
        lng: p.representative_longitude || p.lng,
        year
      }));

    allPlaces = allPlaces.concat(results)

    if(isTooBig && isPlacesEmtpy) {
      break;
    }
    if(i == 1 && isStillTooBig && isPlacesEmtpy) {
      break;
    }
  }

  return allPlaces.filter(row => row.lat && row.lng)
}

function normalizeLng(lng) {
  // Always returns a longitude in [-180, 180]
  const wrapped = ((lng + 360) % 360);
  return wrapped > 180 ? wrapped - 360 : wrapped;
}

function getMapWorldOffset(lng) {
  // Returns which copy of the world we're on
  return Math.floor((lng + 180) / 360);
}

function normalizeBoundsToVisibleWorld(bounds) {
  const isOversized = (bounds[2] - bounds[0]) / 360 > Math.abs(1)
  let lng1 = normalizeLng(bounds[0])
  let lng2 = normalizeLng(bounds[2])
  const worldMinLng = Math.min(lng1, lng2)
  const worldMaxLng = Math.max(lng1, lng2)

  return [
    isOversized ? -180 : worldMinLng,
    Math.max(-90, bounds[1]),
    isOversized ? 180 : worldMaxLng,
    Math.min(90, bounds[3])
  ]
}


  

function placeWithinBounds(p, bounds) {
  return p.lat > bounds[1] && p.lat < bounds[3]
    && p.lng > bounds[0] && p.lng < bounds[2]
}

// Load and parse the CSV file (no headers)
function loadCSV(filePath) {
  if (CACHE[filePath]) {
    return CACHE[filePath]
  }
  const csvData = fs.readFileSync(filePath, "utf8");
  const headers = LOCATIONS[path.basename(filePath)];
  const parsed = Papa.parse(csvData, {
    header: !!headers,
    skipEmptyLines: true,
    dynamicTyping: true,
    transformHeader: (name, index) => headers === true ? name : headers[index]
  });


  CACHE[filePath] = parsed.data
  //console.log(parsed.data)
  // Map raw CSV into structured records
  return parsed.data
}

module.exports = geolocationProvider
